In [1]:
#!pip install simple_fpa

In [2]:
%%capture out
! cd ..; pip install .

In [3]:
print((out.stdout.split('\n')[-2]))

In [4]:
import os
os.cpu_count()

8

In [4]:
from simple_fpa import Model, Simulator
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats import beta, powerlaw

rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Arial"],
    "lines.linewidth": 1,
    "figure.dpi":200
})

import warnings
warnings.filterwarnings("ignore")

import sys
import ujson

In [7]:
smoothing_rate = 0.3
trim_percent = 5
eps = 0.05
frec = {3:1}
draws = 1000 # 3000

In [9]:
d = dict()
sample_sizes = [1000, 10000, 100000]
rvtexts = ['beta(1,1)','beta(2,2)','beta(5,2)',
           'beta(2,5)','powerlaw(2)','powerlaw(3)']

In [9]:
for sample_size in sample_sizes:
    print('*'*30)
    print('sample size:', sample_size)
    d[sample_size] = dict()
    for rvtext in rvtexts:
        print('   random variable:', rvtext)
        d[sample_size][rvtext] = dict()
        rv = eval(rvtext)
        sim = Simulator(sample_size, smoothing_rate, trim_percent, frec, rv.pdf, rv.ppf, eps, draws)
        sim.calibrate()
        sim.simulate()
        for nominal_coverage in [90, 95, 99]:
            coverage = sim.get_coverage(nominal_coverage = nominal_coverage, digits = 3)
            print('      nominal coverage:', nominal_coverage/100, ':', coverage)
            d[sample_size][rvtext][nominal_coverage] = coverage

******************************
sample size: 1000
   random variable: beta(1,1)
      nominal coverage: 0.9 : {'q': 0.936, 'v': 0.937, 'bs': 0.944, 'rev': 0.942, 'ts': 0.924}
      nominal coverage: 0.95 : {'q': 0.978, 'v': 0.973, 'bs': 0.973, 'rev': 0.983, 'ts': 0.973}
      nominal coverage: 0.99 : {'q': 0.999, 'v': 1.0, 'bs': 0.997, 'rev': 0.999, 'ts': 0.995}
   random variable: beta(2,2)
      nominal coverage: 0.9 : {'q': 0.953, 'v': 0.933, 'bs': 0.942, 'rev': 0.941, 'ts': 0.932}
      nominal coverage: 0.95 : {'q': 0.989, 'v': 0.975, 'bs': 0.978, 'rev': 0.979, 'ts': 0.972}
      nominal coverage: 0.99 : {'q': 1.0, 'v': 0.999, 'bs': 0.998, 'rev': 1.0, 'ts': 0.995}
   random variable: beta(5,2)
      nominal coverage: 0.9 : {'q': 0.961, 'v': 0.937, 'bs': 0.949, 'rev': 0.946, 'ts': 0.939}
      nominal coverage: 0.95 : {'q': 0.988, 'v': 0.975, 'bs': 0.977, 'rev': 0.983, 'ts': 0.973}
      nominal coverage: 0.99 : {'q': 0.999, 'v': 1.0, 'bs': 0.998, 'rev': 1.0, 'ts': 0.996}
   random 

In [10]:
filename = './coverage.txt'

In [11]:
file = open(filename, 'w')
file.write(ujson.dumps(d, indent=4))
file.close()

In [11]:
file = open(filename, 'r')
d = ujson.loads(file.read())
file.close()

In [14]:
for coverage in ['90','95','99']:
    with open('./coverage_latex'+'_'+coverage+'.txt', 'w') as file:
        message = "{:11} & {:5} & {:5} & {:5} & {:5} & {:5} \\\\".format('distro', 'q', 'v', 'bs', 'rev', 'ts')
        print(message)
        for sample_size in ['1000','10000','100000']:
            file.write(message+'\n')
            message = '-'*30 +'\n' + 'sample size:' + str(sample_size) + '\n' + '-'*30
            print(message)
            file.write(message+'\n')
            for rvtext in rvtexts:
                message = "{:11} & {:5} & {:5} & {:5} & {:5} & {:5} \\\\".format(rvtext, *[str(v) for k,v in d[sample_size][rvtext][coverage].items()])
                print(message)
                file.write(message+'\n')

distro      & q     & v     & bs    & rev   & ts    \\
------------------------------
sample size:1000
------------------------------
beta(1,1)   & 0.948 & 0.94  & 0.939 & 0.941 & 0.93  \\
beta(2,2)   & 0.96  & 0.939 & 0.941 & 0.935 & 0.93  \\
beta(5,2)   & 0.957 & 0.944 & 0.946 & 0.944 & 0.935 \\
beta(2,5)   & 0.953 & 0.935 & 0.933 & 0.929 & 0.925 \\
powerlaw(2) & 0.958 & 0.948 & 0.947 & 0.952 & 0.934 \\
powerlaw(3) & 0.956 & 0.955 & 0.95  & 0.957 & 0.936 \\
------------------------------
sample size:10000
------------------------------
beta(1,1)   & 0.92  & 0.913 & 0.914 & 0.912 & 0.911 \\
beta(2,2)   & 0.926 & 0.916 & 0.917 & 0.908 & 0.911 \\
beta(5,2)   & 0.922 & 0.916 & 0.921 & 0.914 & 0.913 \\
beta(2,5)   & 0.923 & 0.914 & 0.917 & 0.906 & 0.91  \\
powerlaw(2) & 0.922 & 0.916 & 0.923 & 0.924 & 0.913 \\
powerlaw(3) & 0.923 & 0.922 & 0.925 & 0.926 & 0.915 \\
------------------------------
sample size:100000
------------------------------
beta(1,1)   & 0.905 & 0.9   & 0.903 & 0.907 &